<a href="https://colab.research.google.com/github/SergeyShchus/GIS_Portal/blob/master/Notebook/Colab/HTML_table_to_Pandas_Data_Frame_to_Portal_Item.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

In [6]:
df = pd.read_html("https://en.wikipedia.org/wiki/Number_of_guns_per_capita_by_country")[0]
df.head()

,Unnamed: 0,"Country (or dependent territory, subnational area, etc.)",Estimate of civilian firearms per 100 persons,Region,Subregion,Population 2017,Estimate of firearms in civilian possession,Computation method,Registered firearms,Unregistered firearms,Notes
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,United States,120.5,Americas,North America,326474000.0,393347000,1.0,1073743.0,392273257.0,[note 2]
2,2.0,Falkland Islands,62.1,Americas,South America,3000.0,2000,2.0,1705.0,295.0,NaN
3,3.0,Yemen,52.8,Asia,Western Asia,28120000.0,14859000,2.0,NaN,NaN,NaN
4,4.0,New Caledonia,42.5,Oceania,Melanesia,270000.0,115000,2.0,55000.0,60000.0,NaN


In [7]:
df.dropna(thresh=4, inplace=True)

In [8]:
del df['Unnamed: 0']
del df['Notes']

In [9]:
df.head()

,"Country (or dependent territory, subnational area, etc.)",Estimate of civilian firearms per 100 persons,Region,Subregion,Population 2017,Estimate of firearms in civilian possession,Computation method,Registered firearms,Unregistered firearms
1,United States,120.5,Americas,North America,326474000.0,393347000,1.0,1073743.0,392273257.0
2,Falkland Islands,62.1,Americas,South America,3000.0,2000,2.0,1705.0,295.0
3,Yemen,52.8,Asia,Western Asia,28120000.0,14859000,2.0,NaN,NaN
4,New Caledonia,42.5,Oceania,Melanesia,270000.0,115000,2.0,55000.0,60000.0
5,Montenegro,39.1,Europe,Southern Europe,626000.0,245000,3.0,103536.0,141464.0


In [10]:
df.iloc[0,1] = 120.5

In [11]:
df.dtypes

Country (or dependent territory, subnational area, etc.)     object
Estimate of civilian firearms per 100 persons               float64
Region                                                       object
Subregion                                                    object
Population 2017                                             float64
Estimate of firearms in civilian possession                  object
Computation method                                          float64
Registered firearms                                         float64
Unregistered firearms                                       float64
dtype: object

In [12]:
converted_column = pd.to_numeric(df["Estimate of civilian firearms per 100 persons"], errors = 'coerce')
df['Estimate of civilian firearms per 100 persons'] = converted_column
df.head()

,"Country (or dependent territory, subnational area, etc.)",Estimate of civilian firearms per 100 persons,Region,Subregion,Population 2017,Estimate of firearms in civilian possession,Computation method,Registered firearms,Unregistered firearms
1,United States,120.5,Americas,North America,326474000.0,393347000,1.0,1073743.0,392273257.0
2,Falkland Islands,62.1,Americas,South America,3000.0,2000,2.0,1705.0,295.0
3,Yemen,52.8,Asia,Western Asia,28120000.0,14859000,2.0,NaN,NaN
4,New Caledonia,42.5,Oceania,Melanesia,270000.0,115000,2.0,55000.0,60000.0
5,Montenegro,39.1,Europe,Southern Europe,626000.0,245000,3.0,103536.0,141464.0


In [13]:
!pip install arcgis

In [14]:
from arcgis.gis import GIS
import json

gis = GIS()

In [15]:
fc = gis.content.import_data(df, {"CountryCode":"Country__or_dependent_territory__subnational_area__etc__"})

In [16]:
map1 = gis.map('UK')
map1

MapView(layout=Layout(height='400px', width='100%'))

In [17]:
map1.add_layer(fc, {"renderer":"ClassedSizeRenderer",
                    "field_name": "Estimate_of_civilian_firearms_per_100_persons"})

In [ ]:
item_properties = {
    "title": "Worldwide Firearms ownership",
    "tags" : "guns,violence",
    "snippet": " GSR Worldwide firearms ownership",
    "description": "test description",
    "text": json.dumps({"featureCollection": {"layers": [dict(fc.layer)]}}),
    "type": "Feature Collection",
    "typeKeywords": "Data, Feature Collection, Singlelayer",
    "extent" : "-102.5272,-41.7886,172.5967,64.984"
}

item = gis.content.add(item_properties)

In [ ]:
search_result = gis.content.search("Worldwide Firearms ownership")
search_result[0]